In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading the csv file compiled by joining the 9 tables and Target value.
df=pd.read_csv('final_df.csv')
df = df.drop(columns=['key_2', 'key_1']) #unique identifier removal
df =df.drop(columns=['NumJobsApplicant']) # value = 1 for all the observations.
df =df.drop(columns=['TradeInPctBBVehicleValue', 'TradeInPctLine3', 'TradeInPctLine4', 'TradeInPctLine5']) # dependent on TradeInAmt
df =df.drop(columns=['BackendWarrantyAmt','BackendTotalPctBBVehicleValue', 'BackendTotalPctBBVehicleValueGTE13PctIndicator','BackendTotalPctLine3', 'BackendTotalPctLine4', 'BackendTotalPctLine5']) # dependent on BackendTotAmt)
df =df.drop(columns=['LoanAmtLine3','LoanAmtLine4']) #dependent on FinanceAmt

In [3]:
df.head()

,masked_acct,Vintage,ChargeOffMonthKey,ChargeOffMOB,Loss_Date_150,BalanceAtDefault,JointIndicator,LCPIndicator,FICOScore,StateApplicant,...,NumExceptionDTI,NumExceptionPctAvgNumExceptionLast30DaysBookedLoans,NumPriorCrescentApplications,NumPriorCrescentApplicationsLast30Days,NumPriorCrescentApplicationsLast60Days,NumPriorCrescentApplicationsLast90Days,NumPriorCrescentApplicationsLast180Days,JDPUVIDiffCOFromOrig,MoodysUVIDiffCOFromOrig,true_recovery_rate
0,0001374cd7856b8999ef0b661d515142,201708,201811,15,11/5/2018 0:00,9914.12,0,1,535.0,MO,...,0,1.1903,0,0,0,0,0,5.937,2.39,0.1856
1,000170f205857037065589a606caae25,201803,201811,8,11/26/2018 0:00,22211.39,0,0,652.0,GA,...,0,0.0000,0,0,0,0,0,5.258,1.49,0.5442
2,00060788de8277720911f32a3b355754,201501,201704,27,4/26/2017 0:00,8484.37,0,0,534.0,FL,...,0,0.0000,0,0,0,0,0,-10.699,-7.92,0.0000
3,000b9612a76d2800a8daad266c9e1eb1,201505,201611,18,11/13/2016 0:00,15470.65,0,0,467.0,NM,...,0,1.9463,1,0,1,1,1,-7.571,-7.78,0.3586
4,000c9915e9eb935ee32c691e2f3efb49,201711,201807,8,7/26/2018 0:00,14905.78,0,0,495.0,NC,...,0,0.0000,1,0,0,0,1,0.729,0.56,0.5889


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 78 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   masked_acct                                          13718 non-null  object 
 1   Vintage                                              13718 non-null  int64  
 2   ChargeOffMonthKey                                    13718 non-null  int64  
 3   ChargeOffMOB                                         13718 non-null  int64  
 4   Loss_Date_150                                        13718 non-null  object 
 5   BalanceAtDefault                                     13718 non-null  float64
 6   JointIndicator                                       13718 non-null  int64  
 7   LCPIndicator                                         13718 non-null  int64  
 8   FICOScore                                            9504 non-null

In [124]:
from datetime import datetime
df['Vintage'] = df['Vintage'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))

In [125]:
df['ChargeOffMonthKey'] = df['ChargeOffMonthKey'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))

In [126]:
df['Loss_Date_150'] = df['Loss_Date_150'].astype('datetime64')
df.head()

,masked_acct,Vintage,ChargeOffMonthKey,ChargeOffMOB,Loss_Date_150,BalanceAtDefault,JointIndicator,LCPIndicator,FICOScore,StateApplicant,FICOScorePctAvgFICOLast30DaysBookedLoans,ApplicationWeekday,NewUsedIndicator,VehicleManufacturerRebate,VehicleAge,VehicleMileage,VehicleValueBlackBook,TradeInIndicator,TradeInAmt,VehicleMakeNADA,VehicleModelNADA,JDPowerUsedVehicleIndex,MoodysNumUnemployed,MoodysUsedVehicleIndex,MoodysNumEmployed,MoodysGDPReal,MoodysGasoline,MoodysNumLaborForce,MoodysUnemploymentRate,FinancedAmt,key_1:1,LTVLine3,LTVLine4,LTVLine5,LTVLine3ExclRebate,LTVLine4ExclRebate,LTVLine5ExclRebate,CashDownPmt,CashDownPmtPctBBVehicleValue,CashDownPmtPctLine3,CashDownPmtPctLine4,CashDownPmtPctLine5,PaymentAmt,PaymentToFinancedAmtRatio,PTILine5,BackendGapAmt,BackendTotalAmt,BackendType,TotalDownPmt,TotalDownPmtPctBBVehicleValue,TotalDownPmtPctLine3,TotalDownPmtPctLine4,TotalDownPmtPctLine5,DealerReserve,ContractRate,BuyRate,key_2:1,DealerFeeNet,EmploymentMonthsSinceHiredCurrentEmployerApplicant,EmploymentJobTypeApplicant,CreditBureauNumTrades,CreditBureauNumTradesCurrent,CreditBureauNumCollections,CreditBureauRevolvingCreditLinePctUsed,CreditBureauPctTradesDelq,NumExceptionALL,NumExceptionLTV,NumExceptionPTI,NumExceptionDTI,NumExceptionPctAvgNumExceptionLast30DaysBookedLoans,NumPriorCrescentApplications,NumPriorCrescentApplicationsLast30Days,NumPriorCrescentApplicationsLast60Days,NumPriorCrescentApplicationsLast90Days,NumPriorCrescentApplicationsLast180Days,JDPUVIDiffCOFromOrig,MoodysUVIDiffCOFromOrig,true_recovery_rate
0,0001374cd7856b8999ef0b661d515142,2017-08-01,2018-11-01,15,2018-11-05,9914.12,0,1,535.0,MO,1.0203,4,0,0.0,5,22922,6100.0,0,NaN,FORD,FOCUS,115.357,112.58,136.77,2953.37,278.59,202.69,3065.95,0.0367,10397.4502,ss1239,1.3426,1.3766,1.7045,1.3426,1.3766,1.7045,750.0,0.1230,0.0916,0.0893,0.0721,282.20,0.0271,0.1091,0.0,2000.0,BackendWarranty,750.0,0.1230,0.0916,0.0893,0.0721,440.89,20.75,18.75,ss3548,33.11,101,EMPLOYED,5,0,1,NaN,1.0000,1,1,0,0,1.1903,0,0,0,0,0,5.937,2.39,0.1856
1,000170f205857037065589a606caae25,2018-03-01,2018-11-01,8,2018-11-26,22211.39,0,0,652.0,GA,1.2476,2,0,0.0,4,50523,17575.0,0,NaN,MERCEDES-BENZ,CLA CLASS,116.036,212.79,137.67,4868.79,522.73,237.05,5081.59,0.0419,22360.2500,ss1240,1.0982,1.2214,1.2723,1.0982,1.2214,1.2723,2000.0,0.1138,0.1036,0.0932,0.0894,538.19,0.0241,0.0722,895.0,895.0,BackendGAP,2000.0,0.1138,0.1036,0.0932,0.0894,1054.77,20.20,18.20,ss3549,-780.77,51,EMPLOYED,11,7,0,1.0,0.3636,0,0,0,0,0.0000,0,0,0,0,0,5.258,1.49,0.5442
2,00060788de8277720911f32a3b355754,2015-01-01,2017-04-01,27,2017-04-26,8484.37,0,0,534.0,FL,0.9808,6,0,0.0,2,55703,11225.0,1,2000.0,NISSAN,ALTIMA,124.998,569.99,146.57,9024.72,819.33,238.59,9593.80,0.0594,11444.6602,ss1241,0.9155,1.0196,1.0196,0.9155,1.0196,1.0196,2000.0,0.1782,0.1946,0.1748,0.1748,278.61,0.0243,0.0529,0.0,0.0,BackendNone,4000.0,0.3563,0.3893,0.3495,0.3495,0.00,15.75,15.75,ss3550,398.00,9,EMPLOYED,10,1,5,0.0,0.9000,0,0,0,0,0.0000,0,0,0,0,0,-10.699,-7.92,0.0000
3,000b9612a76d2800a8daad266c9e1eb1,2015-05-01,2016-11-01,18,2016-11-13,15470.65,0,0,467.0,NM,0.8529,7,1,900.0,0,15,14050.0,1,1500.0,KIA,SOUL,125.044,61.33,148.31,874.59,90.92,214.23,935.92,0.0655,16533.8203,ss1242,1.0601,1.1198,1.1768,1.1326,1.1965,1.2573,0.0,0.0000,0.0000,0.0000,0.0000,377.50,0.0228,0.1338,800.0,800.0,BackendGAP,1500.0,0.1068,0.1007,0.0953,0.0907,839.70,17.75,15.75,ss3551,-241.70,12,EMPLOYED,16,2,0,NaN,0.8750,1,0,0,0,1.9463,1,0,1,1,1,-7.571,-7.78,0.3586
4,000c9915e9eb935ee32c691e2f3efb49,2017-11-01,2018-07-01,8,2018-07-26,14905.78,0,0,495.0,NC,0.9435,4,1,2000.0,0,777,10700.0,0,NaN,KIA,RIO,118.470,213.36,137.43,4743.23,489.47,220.40,4956.59,0.0430,15157.8096,ss1243,1.1981,1.3330,1.4166,1.4735,1.6394,1.7423,1200.0,0.1121,0.0936,0.0841,0.0792,347.81,0.0229,0.1546,895.0,895.0,BackendGAP,1200.0,0.1121,0.0936,0.0841,0.0792,0.00,18.00,18.00,ss3552,1124.00,7,OTHER,14,1,11,100.0,0.9286,0,0,0,0,0.0000,1,0,0,0,1,0.729,0.56,0.5889


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 78 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   masked_acct                                          13718 non-null  object        
 1   Vintage                                              13718 non-null  datetime64[ns]
 2   ChargeOffMonthKey                                    13718 non-null  datetime64[ns]
 3   ChargeOffMOB                                         13718 non-null  int64         
 4   Loss_Date_150                                        13718 non-null  datetime64[ns]
 5   BalanceAtDefault                                     13718 non-null  float64       
 6   JointIndicator                                       13718 non-null  int64         
 7   LCPIndicator                                         13718 non-null  int64         
 

In [128]:
#Imputing the data
df['FICOScore']=df['FICOScore'].fillna(df['FICOScore'].mean())
df['FICOScorePctAvgFICOLast30DaysBookedLoans']=df['FICOScorePctAvgFICOLast30DaysBookedLoans'].fillna(df['FICOScorePctAvgFICOLast30DaysBookedLoans'].mean())
df['VehicleValueBlackBook']=df['VehicleValueBlackBook'].fillna(df['VehicleValueBlackBook'].mean())
df['TradeInAmt']=df['TradeInAmt'].fillna(0)


In [129]:
df['TradeInAmt'].info()
df['TradeInAmt'].head()

<class 'pandas.core.series.Series'>
RangeIndex: 13718 entries, 0 to 13717
Series name: TradeInAmt
Non-Null Count  Dtype  
--------------  -----  
13718 non-null  float64
dtypes: float64(1)
memory usage: 107.3 KB


0       0.0
1       0.0
2    2000.0
3    1500.0
4       0.0
Name: TradeInAmt, dtype: float64

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 78 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   masked_acct                                          13718 non-null  object        
 1   Vintage                                              13718 non-null  datetime64[ns]
 2   ChargeOffMonthKey                                    13718 non-null  datetime64[ns]
 3   ChargeOffMOB                                         13718 non-null  int64         
 4   Loss_Date_150                                        13718 non-null  datetime64[ns]
 5   BalanceAtDefault                                     13718 non-null  float64       
 6   JointIndicator                                       13718 non-null  int64         
 7   LCPIndicator                                         13718 non-null  int64         
 

In [131]:
##To display all rows and columns.
# pd.set_option('display.max_columns', None), pd.set_option('display.max_rows', None)
Correlation = df.corr()

In [132]:
Correlation['true_recovery_rate']

ChargeOffMOB                                          -0.309455
BalanceAtDefault                                       0.196177
JointIndicator                                         0.039101
LCPIndicator                                          -0.000347
FICOScore                                             -0.023946
FICOScorePctAvgFICOLast30DaysBookedLoans              -0.015625
ApplicationWeekday                                    -0.005489
NewUsedIndicator                                       0.047394
VehicleManufacturerRebate                              0.066039
VehicleAge                                            -0.026954
VehicleMileage                                        -0.115232
VehicleValueBlackBook                                  0.213596
TradeInIndicator                                       0.051799
TradeInAmt                                             0.043557
JDPowerUsedVehicleIndex                               -0.119778
MoodysNumUnemployed                     

In [133]:
Correlation['true_recovery_rate'].sort_values(ascending = 0)

true_recovery_rate                                     1.000000
MoodysUVIDiffCOFromOrig                                0.265788
VehicleValueBlackBook                                  0.213596
BalanceAtDefault                                       0.196177
FinancedAmt                                            0.152556
JDPUVIDiffCOFromOrig                                   0.146431
PaymentAmt                                             0.133799
BackendTotalAmt                                        0.109226
TotalDownPmt                                           0.099443
CashDownPmt                                            0.069260
VehicleManufacturerRebate                              0.066039
DealerReserve                                          0.062302
TradeInIndicator                                       0.051799
TotalDownPmtPctLine4                                   0.049124
TotalDownPmtPctLine3                                   0.047396
NewUsedIndicator                        

In [136]:
# Identifying the unique values in each columns
unique_values = []
for x in df.columns:
    unique_values.append([x, len(df[x].unique())])
    #prinfting unique values
    print(x ,':', len(df[x].unique()))
pd.DataFrame(unique_values).head(100)

masked_acct : 13718
Vintage : 44
ChargeOffMonthKey : 45
ChargeOffMOB : 46
Loss_Date_150 : 1003
BalanceAtDefault : 13657
JointIndicator : 2
LCPIndicator : 2
FICOScore : 292
StateApplicant : 46
FICOScorePctAvgFICOLast30DaysBookedLoans : 3308
ApplicationWeekday : 7
NewUsedIndicator : 2
VehicleManufacturerRebate : 218
VehicleAge : 14
VehicleMileage : 9046
VehicleValueBlackBook : 879
TradeInIndicator : 2
TradeInAmt : 1262
VehicleMakeNADA : 38
VehicleModelNADA : 214
JDPowerUsedVehicleIndex : 44
MoodysNumUnemployed : 1194
MoodysUsedVehicleIndex : 43
MoodysNumEmployed : 1215
MoodysGDPReal : 1177
MoodysGasoline : 44
MoodysNumLaborForce : 1214
MoodysUnemploymentRate : 368
FinancedAmt : 13250
key_1:1 : 13718
LTVLine3 : 6045
LTVLine4 : 6217
LTVLine5 : 6736
LTVLine3ExclRebate : 6989
LTVLine4ExclRebate : 7193
LTVLine5ExclRebate : 7611
CashDownPmt : 935
CashDownPmtPctBBVehicleValue : 3427
CashDownPmtPctLine3 : 3568
CashDownPmtPctLine4 : 3423
CashDownPmtPctLine5 : 3230
PaymentAmt : 11101
PaymentToFina

,0,1
0,masked_acct,13718
1,Vintage,44
2,ChargeOffMonthKey,45
3,ChargeOffMOB,46
4,Loss_Date_150,1003
5,BalanceAtDefault,13657
6,JointIndicator,2
7,LCPIndicator,2
8,FICOScore,292
9,StateApplicant,46


In [137]:
df['StateApplicant']=df['StateApplicant'].fillna(df['StateApplicant'].mode()[0])

In [138]:
df['VehicleMakeNADA']=df['VehicleMakeNADA'].fillna(df['VehicleMakeNADA'].mode()[0])

In [139]:
## One hot encoding of categorical variables
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'ApplicationWeekday' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df[['ApplicationWeekday']]).toarray())
encoder_df.columns = ['Weekday4', 'Weekday2', 'Weekday6', 'Weekday7','Weekday3', 'Weekday5', 'Weekday1']
print(encoder_df.head())


encoder1 = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'EmploymentJobTypeApplicant' column 
encoder1_df = pd.DataFrame(encoder.fit_transform(df[['EmploymentJobTypeApplicant']]).toarray())
encoder1_df.columns = ['EMPLOYED', 'OTHER', 'RETIRED', 'SELF-EMPLOYED', 'UNEMPLOYED','MILITARY', 'MISSING']
print(encoder1_df.head())


encoder2 = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'StateApplicant' column 
encoder2_df = pd.DataFrame(encoder.fit_transform(df[['StateApplicant']]).toarray())
encoder2_df.columns = ['MO', 'GA', 'FL', 'NM', 'NC', 'IN', 'LA', 'TX', 'NJ', 'NY', 'MI',
       'OK', 'SC', 'MS', 'OH', 'KY', 'VA', 'TN', 'AL', 'PA', 'AZ', 'WA',
       'IL', 'WV', 'CO', 'AR', 'DC', 'NV', 'MN', 'MD', 'KS', 'MA', 
       'ID', 'DE', 'CT', 'UT', 'WY', 'VT', 'ME', 'WI', 'SD', 'OR', 'CA',
       'PR', 'MT', 'IA']
encoder2_df.head()


encoder3 = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'VehicleMakeNADA' column 
encoder3_df = pd.DataFrame(encoder.fit_transform(df[['VehicleMakeNADA']]).toarray())
encoder3_df.columns = ['FORD', 'MERCEDES-BENZ', 'NISSAN', 'KIA', 'CHEVROLET', 
       'MAZDA', 'TOYOTA', 'CHRYSLER', 'DODGE', 'SUBARU', 'HYUNDAI',
       'DODGE TRUCK', 'JEEP', 'BUICK', 'VOLKSWAGEN', 'GMC LIGHT DUTY',
       'BMW', 'FIAT', 'VOLVO', 'FORD TRUCK', 'INFINITI',
       'CHEVROLET TRUCK', 'HONDA', 'MINI', 'MERCURY', 'CADILLAC',
       'LINCOLN', 'JAGUAR', 'MITSUBISHI', 'SUZUKI', 'LEXUS', 'ACURA',
       'PONTIAC', 'SCION', 'AUDI', 'SATURN', 'HUMMER']
encoder3_df.head()


encoder4 = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'BackendType' column 
encoder4_df = pd.DataFrame(encoder.fit_transform(df[['BackendType']]).toarray())
encoder4_df.columns = ['BackendWarranty', 'BackendGAP', 'BackendNone',
       'BackendGapAndWarranty']
encoder4_df.head()

#merge one-hot encoded columns back with original DataFrame
df = df.join(encoder_df)


   Weekday4  Weekday2  Weekday6  Weekday7  Weekday3  Weekday5  Weekday1
0       0.0       0.0       0.0       1.0       0.0       0.0       0.0
1       0.0       1.0       0.0       0.0       0.0       0.0       0.0
2       0.0       0.0       0.0       0.0       0.0       1.0       0.0
3       0.0       0.0       0.0       0.0       0.0       0.0       1.0
4       0.0       0.0       0.0       1.0       0.0       0.0       0.0
   EMPLOYED  OTHER  RETIRED  SELF-EMPLOYED  UNEMPLOYED  MILITARY  MISSING
0       1.0    0.0      0.0            0.0         0.0       0.0      0.0
1       1.0    0.0      0.0            0.0         0.0       0.0      0.0
2       1.0    0.0      0.0            0.0         0.0       0.0      0.0
3       1.0    0.0      0.0            0.0         0.0       0.0      0.0
4       0.0    0.0      0.0            1.0         0.0       0.0      0.0


In [140]:
df = df.join(encoder1_df)

In [141]:
df = df.join(encoder2_df)

In [142]:
df = df.join(encoder3_df)

In [143]:
df = df.join(encoder4_df)

In [144]:
df.head()

,masked_acct,Vintage,ChargeOffMonthKey,ChargeOffMOB,Loss_Date_150,BalanceAtDefault,JointIndicator,LCPIndicator,FICOScore,StateApplicant,FICOScorePctAvgFICOLast30DaysBookedLoans,ApplicationWeekday,NewUsedIndicator,VehicleManufacturerRebate,VehicleAge,VehicleMileage,VehicleValueBlackBook,TradeInIndicator,TradeInAmt,VehicleMakeNADA,VehicleModelNADA,JDPowerUsedVehicleIndex,MoodysNumUnemployed,MoodysUsedVehicleIndex,MoodysNumEmployed,MoodysGDPReal,MoodysGasoline,MoodysNumLaborForce,MoodysUnemploymentRate,FinancedAmt,key_1:1,LTVLine3,LTVLine4,LTVLine5,LTVLine3ExclRebate,LTVLine4ExclRebate,LTVLine5ExclRebate,CashDownPmt,CashDownPmtPctBBVehicleValue,CashDownPmtPctLine3,CashDownPmtPctLine4,CashDownPmtPctLine5,PaymentAmt,PaymentToFinancedAmtRatio,PTILine5,BackendGapAmt,BackendTotalAmt,BackendType,TotalDownPmt,TotalDownPmtPctBBVehicleValue,TotalDownPmtPctLine3,TotalDownPmtPctLine4,TotalDownPmtPctLine5,DealerReserve,ContractRate,BuyRate,key_2:1,DealerFeeNet,EmploymentMonthsSinceHiredCurrentEmployerApplicant,EmploymentJobTypeApplicant,CreditBureauNumTrades,CreditBureauNumTradesCurrent,CreditBureauNumCollections,CreditBureauRevolvingCreditLinePctUsed,CreditBureauPctTradesDelq,NumExceptionALL,NumExceptionLTV,NumExceptionPTI,NumExceptionDTI,NumExceptionPctAvgNumExceptionLast30DaysBookedLoans,NumPriorCrescentApplications,NumPriorCrescentApplicationsLast30Days,NumPriorCrescentApplicationsLast60Days,NumPriorCrescentApplicationsLast90Days,NumPriorCrescentApplicationsLast180Days,JDPUVIDiffCOFromOrig,MoodysUVIDiffCOFromOrig,true_recovery_rate,Weekday4,Weekday2,Weekday6,Weekday7,Weekday3,Weekday5,Weekday1,EMPLOYED,OTHER,RETIRED,SELF-EMPLOYED,UNEMPLOYED,MILITARY,MISSING,MO,GA,FL,NM,NC,IN,LA,TX,NJ,NY,MI,OK,SC,MS,OH,KY,VA,TN,AL,PA,AZ,WA,IL,WV,CO,AR,DC,NV,MN,MD,KS,MA,ID,DE,CT,UT,WY,VT,ME,WI,SD,OR,CA,PR,MT,IA,FORD,MERCEDES-BENZ,NISSAN,KIA,CHEVROLET,MAZDA,TOYOTA,CHRYSLER,DODGE,SUBARU,HYUNDAI,DODGE TRUCK,JEEP,BUICK,VOLKSWAGEN,GMC LIGHT DUTY,BMW,FIAT,VOLVO,FORD TRUCK,INFINITI,CHEVROLET TRUCK,HONDA,MINI,MERCURY,CADILLAC,LINCOLN,JAGUAR,MITSUBISHI,SUZUKI,LEXUS,ACURA,PONTIAC,SCION,AUDI,SATURN,HUMMER,BackendWarranty,BackendGAP,BackendNone,BackendGapAndWarranty
0,0001374cd7856b8999ef0b661d515142,2017-08-01,2018-11-01,15,2018-11-05,9914.12,0,1,535.0,MO,1.0203,4,0,0.0,5,22922,6100.0,0,0.0,FORD,FOCUS,115.357,112.58,136.77,2953.37,278.59,202.69,3065.95,0.0367,10397.4502,ss1239,1.3426,1.3766,1.7045,1.3426,1.3766,1.7045,750.0,0.1230,0.0916,0.0893,0.0721,282.20,0.0271,0.1091,0.0,2000.0,BackendWarranty,750.0,0.1230,0.0916,0.0893,0.0721,440.89,20.75,18.75,ss3548,33.11,101,EMPLOYED,5,0,1,NaN,1.0000,1,1,0,0,1.1903,0,0,0,0,0,5.937,2.39,0.1856,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,000170f205857037065589a606caae25,2018-03-01,2018-11-01,8,2018-11-26,22211.39,0,0,652.0,GA,1.2476,2,0,0.0,4,50523,17575.0,0,0.0,MERCEDES-BENZ,CLA CLASS,116.036,212.79,137.67,4868.79,522.73,237.05,5081.59,0.0419,22360.2500,ss1240,1.0982,1.2214,1.2723,1.0982,1.2214,1.2723,2000.0,0.1138,0.1036,0.0932,0.0894,538.19,0.0241,0.0722,895.0,895.0,BackendGAP,2000.0,0.1138,0.1036,0.0932,0.0894,1054.77,20.20,18.20,ss3549,-780.77,51,EMPLOYED,11,7,0,1.0,0.3636,0,0,0,0,0.0000,0,0,0,0,0,5.258,1.49,0.5442,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,00060788de8277720911f32a3b355754,2015-01-01,2017-04-01

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Columns: 179 entries, masked_acct to BackendGapAndWarranty
dtypes: datetime64[ns](3), float64(147), int64(21), object(8)
memory usage: 18.7+ MB


In [146]:
df['MoodysNumEmployed']=df['MoodysNumEmployed'].fillna(df['MoodysNumEmployed'].median())
df['MoodysNumUnemployed']=df['MoodysNumUnemployed'].fillna(df['MoodysNumUnemployed'].median())
df['MoodysGDPReal']=df['MoodysGDPReal'].fillna(df['MoodysGDPReal'].median())
df['MoodysNumLaborForce']=df['MoodysNumLaborForce'].fillna(df['MoodysNumLaborForce'].median())
df['MoodysUnemploymentRate']=df['MoodysUnemploymentRate'].fillna(df['MoodysUnemploymentRate'].median())
df['LTVLine3']=df['LTVLine3'].fillna(df['LTVLine3'].median())
df['LTVLine4']=df['LTVLine4'].fillna(df['LTVLine4'].median())
df['LTVLine5']=df['LTVLine5'].fillna(df['LTVLine5'].median())
df['LTVLine3ExclRebate']=df['LTVLine3ExclRebate'].fillna(df['LTVLine3ExclRebate'].median())
df['LTVLine4ExclRebate']=df['LTVLine4ExclRebate'].fillna(df['LTVLine4ExclRebate'].median())
df['LTVLine5ExclRebate']=df['LTVLine5ExclRebate'].fillna(df['LTVLine5ExclRebate'].median())
df['CashDownPmtPctBBVehicleValue']=df['CashDownPmtPctBBVehicleValue'].fillna(df['CashDownPmtPctBBVehicleValue'].median())
df['TotalDownPmtPctBBVehicleValue']=df['TotalDownPmtPctBBVehicleValue'].fillna(df['TotalDownPmtPctBBVehicleValue'].median())
df['TotalDownPmtPctBBVehicleValue']=df['TotalDownPmtPctBBVehicleValue'].fillna(df['TotalDownPmtPctBBVehicleValue'].median())
df['CreditBureauPctTradesDelq']=df['CreditBureauPctTradesDelq'].fillna(df['CreditBureauPctTradesDelq'].median())
df['NumExceptionPctAvgNumExceptionLast30DaysBookedLoans']=df['NumExceptionPctAvgNumExceptionLast30DaysBookedLoans'].fillna(df['NumExceptionPctAvgNumExceptionLast30DaysBookedLoans'].median())

In [147]:
# df.isnull().sum()
pd.isnull(df).sum()[pd.isnull(df).sum() > 0]

VehicleModelNADA                           2523
CreditBureauRevolvingCreditLinePctUsed    10067
dtype: int64

In [181]:
# Split the dataset
from sklearn.model_selection import train_test_split

X = df.drop(columns = ['true_recovery_rate', 'masked_acct', 'Vintage', 'Loss_Date_150','ChargeOffMonthKey', 'VehicleModelNADA','BackendType', 'StateApplicant', 'VehicleMakeNADA','EmploymentJobTypeApplicant','ApplicationWeekday','key_1:1','key_2:1','CreditBureauRevolvingCreditLinePctUsed'])
y = df['true_recovery_rate']

# multiple outputs, multiple variables

X_train, X_test, y_train, y_test = train_test_split( X , y  , 
                                                    test_size = 0.3, random_state = 22 )




In [182]:
X.head()

,ChargeOffMOB,BalanceAtDefault,JointIndicator,LCPIndicator,FICOScore,FICOScorePctAvgFICOLast30DaysBookedLoans,NewUsedIndicator,VehicleManufacturerRebate,VehicleAge,VehicleMileage,VehicleValueBlackBook,TradeInIndicator,TradeInAmt,JDPowerUsedVehicleIndex,MoodysNumUnemployed,MoodysUsedVehicleIndex,MoodysNumEmployed,MoodysGDPReal,MoodysGasoline,MoodysNumLaborForce,MoodysUnemploymentRate,FinancedAmt,LTVLine3,LTVLine4,LTVLine5,LTVLine3ExclRebate,LTVLine4ExclRebate,LTVLine5ExclRebate,CashDownPmt,CashDownPmtPctBBVehicleValue,CashDownPmtPctLine3,CashDownPmtPctLine4,CashDownPmtPctLine5,PaymentAmt,PaymentToFinancedAmtRatio,PTILine5,BackendGapAmt,BackendTotalAmt,TotalDownPmt,TotalDownPmtPctBBVehicleValue,TotalDownPmtPctLine3,TotalDownPmtPctLine4,TotalDownPmtPctLine5,DealerReserve,ContractRate,BuyRate,DealerFeeNet,EmploymentMonthsSinceHiredCurrentEmployerApplicant,CreditBureauNumTrades,CreditBureauNumTradesCurrent,CreditBureauNumCollections,CreditBureauPctTradesDelq,NumExceptionALL,NumExceptionLTV,NumExceptionPTI,NumExceptionDTI,NumExceptionPctAvgNumExceptionLast30DaysBookedLoans,NumPriorCrescentApplications,NumPriorCrescentApplicationsLast30Days,NumPriorCrescentApplicationsLast60Days,NumPriorCrescentApplicationsLast90Days,NumPriorCrescentApplicationsLast180Days,JDPUVIDiffCOFromOrig,MoodysUVIDiffCOFromOrig,Weekday4,Weekday2,Weekday6,Weekday7,Weekday3,Weekday5,Weekday1,EMPLOYED,OTHER,RETIRED,SELF-EMPLOYED,UNEMPLOYED,MILITARY,MISSING,MO,GA,FL,NM,NC,IN,LA,TX,NJ,NY,MI,OK,SC,MS,OH,KY,VA,TN,AL,PA,AZ,WA,IL,WV,CO,AR,DC,NV,MN,MD,KS,MA,ID,DE,CT,UT,WY,VT,ME,WI,SD,OR,CA,PR,MT,IA,FORD,MERCEDES-BENZ,NISSAN,KIA,CHEVROLET,MAZDA,TOYOTA,CHRYSLER,DODGE,SUBARU,HYUNDAI,DODGE TRUCK,JEEP,BUICK,VOLKSWAGEN,GMC LIGHT DUTY,BMW,FIAT,VOLVO,FORD TRUCK,INFINITI,CHEVROLET TRUCK,HONDA,MINI,MERCURY,CADILLAC,LINCOLN,JAGUAR,MITSUBISHI,SUZUKI,LEXUS,ACURA,PONTIAC,SCION,AUDI,SATURN,HUMMER,BackendWarranty,BackendGAP,BackendNone,BackendGapAndWarranty
0,15,9914.12,0,1,535.0,1.0203,0,0.0,5,22922,6100.0,0,0.0,115.357,112.58,136.77,2953.37,278.59,202.69,3065.95,0.0367,10397.4502,1.3426,1.3766,1.7045,1.3426,1.3766,1.7045,750.0,0.1230,0.0916,0.0893,0.0721,282.20,0.0271,0.1091,0.0,2000.0,750.0,0.1230,0.0916,0.0893,0.0721,440.89,20.75,18.75,33.11,101,5,0,1,1.0000,1,1,0,0,1.1903,0,0,0,0,0,5.937,2.39,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8,22211.39,0,0,652.0,1.2476,0,0.0,4,50523,17575.0,0,0.0,116.036,212.79,137.67,4868.79,522.73,237.05,5081.59,0.0419,22360.2500,1.0982,1.2214,1.2723,1.0982,1.2214,1.2723,2000.0,0.1138,0.1036,0.0932,0.0894,538.19,0.0241,0.0722,895.0,895.0,2000.0,0.1138,0.1036,0.0932,0.0894,1054.77,20.20,18.20,-780.77,51,11,7,0,0.3636,0,0,0,0,0.0000,0,0,0,0,0,5.258,1.49,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,27,8484.37,0,0,534.0,0.9808,0,0.0,2,55703,11225.0,1,2000.0,124.998,569.99,146.57,9024.72,819.33,238.59,9593.80,0.0594,11444.6602,0.9155,1.0196,1.0196,0.9155,1.0196,1.0196,2000.0,0.1782,0.1946,0.1748,0.1748,278.61,0.0243,0.0529,0.0,0.0,4000.0,0.3563,0.3893,0.3495,0.3495,0.00,15.75,15.75,398.00,9,10,1,5,0.9000,0,0,0,0,0.0000,0,0,0,0,0,-10.699,-7.92,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [183]:
X_train.shape, X_test.shape, y_train.shape

((9602, 165), (4116, 165), (9602,))

In [184]:
from sklearn.linear_model import LinearRegression

my_model = LinearRegression()
my_model.fit(X_train, y_train)

LinearRegression()

In [185]:
my_model.coef_, my_model.intercept_

(array([-1.02242465e-02, -3.67359137e-05,  2.75635192e-02, -8.67509311e-03,
        -1.97937713e-04,  1.05105618e-01, -2.93695062e-02,  4.16838515e-06,
        -9.86707592e-05, -1.14356873e-06,  1.79123397e-06,  2.77285962e-02,
        -1.89711494e-06, -1.76863357e-03, -2.83476105e-03,  4.50501358e-03,
        -3.18132322e-03,  3.17544592e-04,  1.13481898e-04,  3.14092520e-03,
        -2.34672523e+00,  1.36893667e-05, -1.22914878e-01,  1.41974247e-01,
        -2.22510255e-01,  1.38073470e-01, -3.66143560e-01,  2.34980153e-01,
         3.07980428e-06,  2.59082334e-02,  4.27442666e-01, -9.69161083e-01,
         4.68785630e-01,  1.06364501e-03, -2.11297541e+01,  1.29375811e-01,
         6.11465067e-06, -1.35402320e-05,  1.18283219e-06, -4.55299477e-03,
        -3.89870514e-01,  7.18977959e-01, -2.26129124e-01, -4.36564849e-05,
         1.96026631e-02, -1.63178946e-02, -1.59629675e-05,  6.49983075e-05,
         1.86857753e-04,  1.38772907e-03,  3.42453512e-04,  1.58032950e-02,
        -1.6

In [186]:
y_pred_test = my_model.predict(X_test)

y_pred_test.shape

(4116,)

In [187]:
# MAE

## get errors
e = y_test - y_pred_test

## absolute, mean
MAE = np.sum(np.abs(e))/y_test.shape
np.mean(np.abs(e))

# MSE

MSE = np.mean(e**2)

# RMSE

RMSE = MSE**0.5

MAE, MSE, RMSE

(array([0.15923977]), 0.042599680317877794, 0.20639689997157853)

In [188]:
my_model.score(X_test,y_test),my_model.score(X_train,y_train)

(0.20553750261446857, 0.23914603571018966)

In [189]:
with open('lr_model_pickle', 'wb') as f:
    pickle.dump(model,f)

In [190]:
with open('lr_model_pickle', 'rb')as f:
    LR = pickle.load(f)

In [191]:
pickle_lr_test_pred = LR.predict(X_test)

In [192]:
#LASSO

In [193]:

# # Model and prediction with default hyperparameter

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# Grid Search with CV - LASSO Case
lasso_grid = { 'alpha' : [0.0001]}

# 2. Define function and fit the data
# EST
lasso = Lasso()

lasso_cv_reg = GridSearchCV(lasso, lasso_grid, cv = 5)

lasso_cv_reg.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(), param_grid={'alpha': [0.0001]})

In [194]:
lasso_cv_reg.best_params_,lasso_cv_reg.best_score_,lasso_cv_reg.score(X_test,y_test)

({'alpha': 0.0001}, 0.21097359351538286, 0.2083122986831697)

In [195]:
# error
y_test_L1 = lasso_cv_reg.predict(X_test)
e = y_test - y_test_L1
MSE_l1 = np.mean(e**2)

# RMSE

RMSE_l1 = MSE_lp**0.5

RMSE_l1

0.21838974922452178

In [196]:
with open('L1lr_model_pickle', 'wb') as f:
    pickle.dump(model,f)

In [197]:
with open('L1lr_model_pickle', 'rb')as f:
    Lasso = pickle.load(f)

In [198]:
pickle_lasso_test_pred = Lasso.predict(X_test)

In [199]:
####polynomial

In [200]:
# from sklearn.preprocessing  import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# param_poly = {'polynomialfeatures__degree' :  range(1,4)  }
# pipe_poly = PolynomialFeatures()
# lasso_grid = { 'alpha' : [0.0001]}
# lasso = Lasso()
# pipe_lasr = GridSearchCV(lasso, lasso_grid, cv = 5)
# polyreg_pipe = make_pipeline(pipe_poly, pipe_lasr)
# grid_poly = GridSearchCV(polyreg_pipe, param_poly, cv = 5)
# grid_poly.fit(X_train, y_train)

In [201]:
# y_test_grid_poly = grid_poly.predict(X_test)

# #MAE

# MAE = np.mean(np.abs(y_test- y_test_grid_pred))
# MAE

In [202]:
# grid_poly.best_score_

In [203]:
# grid_poly.best_params_

In [204]:
# # error

# e = y_test - y_test_grid_poly
# MSE_lp = np.mean(e**2)

# # RMSE

# RMSE_lp = MSE_lp**0.5

# RMSE_lp

In [205]:
# !pip install xgboost

In [206]:
import xgboost as xgb
model = xgb.XGBRegressor()
xgb_model = model.fit(X_train, y_train)

In [207]:
y_test_predxgb = xgb_model.predict(X_test)
# error

e = y_test - y_test_predxgb
MSE_lp = np.mean(e**2)

# RMSE

RMSE_xgb = MSE_lp**0.5

print('RMSE for XGBooster model is:', RMSE_xgb)

RMSE for XGBooster model is: 0.21838974922452178


In [208]:
RMSE_xgb**2

0.047694082566349515

In [209]:
import pickle

In [210]:
with open('xgb_model_pickle', 'wb') as f:
    pickle.dump(model,f)

In [211]:
with open('xgb_model_pickle', 'rb')as f:
    XGB = pickle.load(f)

In [212]:
pickle_xgb_test_pred = XGB.predict(X_test)

In [213]:
e1 = y_test - pickle_lr_test_pred
e2 = y_test - pickle_lasso_test_pred
e3 = y_test - pickle_xgb_test_pred

MSE_LR = np.mean(e1**2)
MSE_Lasso = np.mean(e2**2)
MSE_XGB = np.mean(e3**2)

# RMSE

RMSE_LR = MSE_LR**0.5
RMSE_Lasso = MSE_Lasso**0.5
RMSE_XGB = MSE_XGB**0.5

print('The RMSE for Linear Regression Model is:', RMSE_LR)
print('The RMSE for L1 Linear Regression Model (Lasso) is:', RMSE_Lasso)
print('The RMSE for XGBoost Model is:', RMSE_XGB)

The RMSE for Linear Regression Model is: 0.21838974922452178
The RMSE for L1 Linear Regression Model (Lasso) is: 0.21838974922452178
The RMSE for XGBoost Model is: 0.21838974922452178
